In [ ]:
%pip install -qU langchain-openai langchain langgraph langchain_community langgraph-supervisor langchain-chroma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.4/152.4 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 57.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.9/438.9 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.8 MB/s eta 0:00:0

In [ ]:
from pydantic import BaseModel
from langchain_openai import ChatOpenAI
from google.colab import userdata
from langgraph.prebuilt import create_react_agent
from langgraph_supervisor import create_supervisor
from langchain_core.tools import tool
from langchain_core.messages import (
    HumanMessage,
)
from langgraph.prebuilt import create_react_agent
import requests
key = userdata.get('OPENAI_API_KEY')

In [ ]:
model = ChatOpenAI(api_key=key,model="gpt-4o-mini")

In [ ]:
from langchain.tools import tool
from datetime import datetime, timedelta
import random


In [ ]:
llm = ChatOpenAI(api_key=key,model="gpt-4o-mini", temperature=0.7)


@tool
def generate_wind_down_routine(sleep_quality: str) -> str:
    """
    Generates a creative, personalized wind-down routine using LLM based on sleep quality: 'good', 'average', or 'poor'.
    """
    prompt = f"""
    I'm designing a nighttime wind-down routine for someone whose sleep quality is "{sleep_quality}".
    Please generate a detailed, personalized routine that includes:
    - Creative, non-repetitive ideas beyond standard suggestions
    - Activities that suit the given sleep quality
    - Natural language phrasing (like a bedtime coach talking to the user)
    Make it sound calming and human-like.
    """

    response = llm.invoke(prompt)
    return response.content


In [ ]:
generate_wind_down_routine.run("average")

"**Nighttime Wind-Down Routine for Enhancing Sleep Quality**\n\nHey there! I'm so glad you're taking the time to craft a soothing nighttime routine. It’s a wonderful way to signal to your body that it's time to unwind and prepare for a restful night's sleep. Let’s create a calming experience that feels personal and refreshing, ensuring you drift off feeling calm and collected.\n\n### **1. **Gentle Transition (1 hour before bed)**\n\nAs the evening settles in, allow yourself to transition from the busyness of the day. Dim the lights in your space to create a cozy ambiance. If you have a favorite scented candle or essential oil (like lavender or chamomile), light it or diffuse it to begin filling your space with calming aromas.\n\n### **2. **Unplug and Reflect (45 minutes before bed)**\n\nTake a moment to unplug from screens. Instead of scrolling through your phone or watching TV, grab a notebook or a journal. Spend 10-15 minutes jotting down your thoughts. You can reflect on three thing

In [ ]:
@tool
def log_sleep(start_time: str, end_time: str) -> str:
    """
    Logs sleep session using start and end timestamps (in ISO format).
    """
    start = datetime.fromisoformat(start_time)
    end = datetime.fromisoformat(end_time)
    duration = (end - start).total_seconds() / 3600
    return f"Sleep logged from {start.strftime('%I:%M %p')} to {end.strftime('%I:%M %p')} ({duration:.2f} hours)."


In [ ]:
from datetime import datetime, timedelta

# llm = ChatOpenAI(temperature=0.5)
llm = ChatOpenAI(api_key=key,model="gpt-4o-mini", temperature=0.5)
@tool
def get_sleep_reminders(bedtime: str) -> list:
    """
    Uses an LLM to generate creative, human-like reminders for winding down before sleep.
    Input: bedtime as a string in 24-hour HH:MM format (e.g., '22:30')
    Output: list of tips tailored to the bedtime.
    """
    try:
        bed_time = datetime.strptime(bedtime, "%H:%M")
    except ValueError:
        return ["Invalid time format. Please use HH:MM in 24-hour format (e.g., 22:30)."]

    # Calculate key times
    caffeine_cutoff = (bed_time - timedelta(hours=6)).strftime("%I:%M %p")
    screen_off_time = (bed_time - timedelta(hours=1)).strftime("%I:%M %p")
    wind_down_time = (bed_time - timedelta(minutes=30)).strftime("%I:%M %p")

    # Prompt to send to the LLM
    prompt = f"""
You are a friendly sleep coach. A user is planning to sleep at {bed_time.strftime('%I:%M %p')}.
Please provide 3 to 5 varied and creative reminders to help them prepare for bed, based on the following:
- Avoid caffeine after {caffeine_cutoff}
- Turn off screens by {screen_off_time}
- Begin wind-down activities by {wind_down_time}

Make the reminders sound natural and motivational — like a bedtime assistant helping them unwind. Avoid repeating generic advice and be warm, gentle, and inventive in your suggestions. Return only a list of reminders (no explanation).
"""

    response = llm.invoke(prompt)
    return response.content.strip().split("\n")


In [ ]:
get_sleep_reminders.run("22:30")


['1. 🌙 As the clock nears 9:30 PM, let’s dim those screens and invite the soft glow of relaxation into your evening. Your dreams will thank you for it!',
 '',
 '2. ☕ Remember, the caffeine fairy bids adieu after 4:30 PM! Sip on a cozy herbal tea instead, and let its warmth wrap you in a soothing embrace.',
 '',
 '3. 🕙 At 10:00 PM, let’s slip into a world of calm. Grab your favorite book or journal, and let your thoughts drift like clouds in the night sky.',
 '',
 '4. 🎶 As the hour approaches, consider putting on some gentle music or nature sounds. It’s the perfect soundtrack to your winding-down ritual!',
 '',
 '5. 🌌 Take a moment to reflect on your day with gratitude. As you prepare for sleep, let go of any lingering thoughts and make space for sweet dreams to dance in!']

In [ ]:
from datetime import datetime
from langchain.tools import tool
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(api_key=key, model="gpt-4o-mini", temperature=0.6)

dummy_sleep_history = [
    {"date": "2025-06-10", "hours_slept": 5.5, "interruptions": 3},
    {"date": "2025-06-11", "hours_slept": 6.0, "interruptions": 2},
    {"date": "2025-06-12", "hours_slept": 5.0, "interruptions": 4},
    {"date": "2025-06-13", "hours_slept": 6.5, "interruptions": 1},
    {"date": "2025-06-14", "hours_slept": 5.8, "interruptions": 3}
]

@tool
def generate_routine_from_history(avg_sleep_hours: float, interruptions: int) -> str:
    """
    Uses an LLM to generate a creative bedtime routine based on sleep history averages.
    """
    prompt = f"""
You are a sleep optimization assistant.

A user has had the following sleep pattern over the last few days:
- Average sleep duration: {avg_sleep_hours:.1f} hours
- Average interruptions per night: {interruptions}

Based on this, suggest a detailed and creative new bedtime routine to help improve their sleep.
The routine should include:
- What time to start winding down
- At least 3 specific, calming activities
- Gentle phrasing (not robotic)
- If the sleep is already good, offer tips for consistency

Avoid repeating generic advice. Make it feel personalized and human.
"""
    response = llm.invoke(prompt)
    return response.content.strip()

def analyze_sleep_history(history: list) -> tuple:
    avg_hours = sum(d["hours_slept"] for d in history) / len(history)
    avg_interruptions = sum(d["interruptions"] for d in history) / len(history)
    return avg_hours, int(round(avg_interruptions))  # ✅ cast to int

# Generate routine
avg_hours, avg_interrupts = analyze_sleep_history(dummy_sleep_history)
routine = generate_routine_from_history.run({
    "avg_sleep_hours": avg_hours,
    "interruptions": avg_interrupts
})
print(routine)


It sounds like you’re ready to embrace a more restful night’s sleep! Let’s create a cozy and calming bedtime routine that can help you transition smoothly into slumber, enhancing both the quality and duration of your rest.

### Winding Down: 9:00 PM

Start winding down around 9:00 PM. This gives you ample time to let your mind and body transition from the busyness of the day to a more tranquil state. 

### Calming Activities to Include:

1. **Gentle Yoga or Stretching (9:00 PM - 9:15 PM)**
   Begin with some gentle yoga or stretching. Focus on deep, slow breaths as you move through each pose. Consider poses like Child’s Pose or Legs-Up-the-Wall to release tension. This not only relaxes your muscles but also calms your mind, preparing you for sleep.

2. **Mindful Journaling (9:15 PM - 9:30 PM)**
   After your yoga, take a few moments to write in a journal. Reflect on your day, noting down any thoughts or feelings that come to mind. You might write three things you’re grateful for or jot

In [ ]:
from langchain.tools import tool
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(api_key=key, model="gpt-4o-mini", temperature=0.3)

@tool
def check_sleep_disorder(avg_sleep: float, latency: float, disturbances: int) -> str:
    """
    Uses an LLM to assess possible sleep disorders based on:
    - avg_sleep: average hours of sleep
    - latency: average time to fall asleep (in minutes)
    - disturbances: number of wake-ups per night

    Returns a natural-language interpretation with possible conditions and lifestyle tips.
    """

    prompt = f"""
You are a sleep health expert. A user has shared the following sleep metrics:
- Average sleep per night: {avg_sleep:.1f} hours
- Time to fall asleep (latency): {latency:.0f} minutes
- Number of wake-ups per night: {disturbances}

Please assess if they may be experiencing any sleep disorders or sleep hygiene issues.
If relevant, mention:
- Risk of disorders (e.g. insomnia, delayed sleep phase disorder, sleep apnea)
- Mild sleep hygiene concerns
- Recommendations (lifestyle, behavioral, or medical advice)

The tone should be supportive and educational. Avoid sounding alarmist.
Return a concise paragraph.
"""

    response = llm.invoke(prompt)
    return response.content.strip()


In [ ]:
result = check_sleep_disorder.run({
    "avg_sleep": 4.8,
    "latency": 60,
    "disturbances": 5
})
print(result)


Based on the sleep metrics you've shared, it appears you may be experiencing some challenges that could indicate potential sleep disorders, such as insomnia, characterized by difficulty falling asleep and maintaining sleep, or possibly delayed sleep phase disorder if your sleep schedule is misaligned with typical patterns. Additionally, waking up five times a night can disrupt your overall sleep quality. While your average sleep duration is below the recommended 7-9 hours for adults, this could also suggest mild sleep hygiene concerns, such as irregular sleep schedules or pre-sleep activities that may be affecting your ability to wind down. To improve your sleep, consider establishing a consistent sleep routine, creating a calming pre-sleep environment, and limiting screen time before bed. If these strategies don’t help, or if you continue to struggle with sleep, it may be beneficial to consult a healthcare professional for further evaluation and support. Remember, small changes can le

In [ ]:
from datetime import datetime, timedelta
from langchain.tools import tool
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(api_key=key, model="gpt-4o-mini", temperature=0.5)

# Optional dummy data
dummy_user_profile = {
    "name": "Alex",
    "chronotype": "night owl",  # Options: morning lark, night owl, balanced
    "recent_sleep_hours": [6.5, 7.0, 5.8, 6.2, 7.3],  # for context
    "prefers_evening_routines": True
}

@tool
def suggest_bedtime(wake_up_time: str, sleep_cycle_hours: float = 7.5) -> str:
    """
    Uses an LLM to suggest a natural bedtime, based on desired sleep cycle and wake-up time,
    including personalized language and behavioral context.
    """

    try:
        wake = datetime.strptime(wake_up_time, "%H:%M")
    except ValueError:
        return "Invalid time format. Please use HH:MM (24-hour format)."

    # Compute bedtime
    bedtime = wake - timedelta(hours=sleep_cycle_hours)
    bedtime_str = bedtime.strftime('%I:%M %p')

    # Contextual prompt
    prompt = f"""
You are a sleep assistant helping a user plan their night. Their profile:
- Name: {dummy_user_profile['name']}
- Chronotype: {dummy_user_profile['chronotype']}
- Preferred evening routine: {dummy_user_profile['prefers_evening_routines']}
- Recent sleep durations: {dummy_user_profile['recent_sleep_hours']}

They want to wake up at {wake_up_time} and aim for {sleep_cycle_hours} hours of sleep.
Suggest a bedtime in a friendly and encouraging tone, including:
- The suggested bedtime ({bedtime_str})
- Why it's a good choice
- A helpful tip to start winding down
Return a short paragraph.
"""

    response = llm.invoke(prompt)
    return response.content.strip()


In [ ]:
print(suggest_bedtime.run({
    "wake_up_time": "06:30",  # 6:30 AM
    "sleep_cycle_hours": 7.5
}))


Hey Alex! To get your desired 7.5 hours of sleep and wake up feeling refreshed at 6:30 AM, I suggest you aim for a bedtime of 11:00 PM. This gives you a solid window to unwind and prepare for a restful night, which is especially important for a night owl like you. To help you start winding down, consider setting aside some time for a relaxing activity, like reading a book or listening to calming music, about an hour before bed. This will signal to your body that it’s time to transition to sleep. Sweet dreams!


In [ ]:
from datetime import datetime, timedelta
from langchain.tools import tool
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(api_key=key, model="gpt-4o-mini", temperature=0.75)

# Dummy recent sleep history (hours slept each night)
dummy_sleep_history = [5.5, 6.0, 4.8, 7.0, 6.5, 5.0]

@tool
def build_routine_with_calendar(next_day_meeting: str, sleep_deficit: float) -> str:
    """
    Builds a personalized next-day wind-down routine considering early meeting time and sleep deficit.
    """
    try:
        meeting_time = datetime.strptime(next_day_meeting, "%H:%M")
    except ValueError:
        return "Invalid time format for meeting. Please use HH:MM in 24-hour format."

    # Compute average recent sleep hours from dummy data
    avg_sleep = sum(dummy_sleep_history) / len(dummy_sleep_history)

    prompt = f"""
You are a helpful sleep coach.

A user has an early meeting at {meeting_time.strftime('%I:%M %p')} tomorrow.
Their recent average sleep duration is {avg_sleep:.1f} hours.
They currently have a sleep deficit of {sleep_deficit:.1f} hours.

Based on this, suggest a personalized wind-down time and bedtime routine for tonight.
Explain why this timing is important and offer gentle advice on how to prepare.
Make the tone supportive and encouraging.
"""

    response = llm.invoke(prompt)
    return response.content.strip()


In [ ]:
print(build_routine_with_calendar.run({
    "next_day_meeting": "08:30",
    "sleep_deficit": 3.2
}))


It sounds like you’re in need of a good night’s sleep before your early meeting tomorrow! Given your current sleep situation, let’s work together to create a calming wind-down routine that will help you feel rested and ready for the day ahead.

### Suggested Wind-Down Time: 9:30 PM

### Bedtime: 10:00 PM

**Why This Timing is Important:**
1. **Wind Down Period:** By starting your wind-down routine at 9:30 PM, you’ll give yourself about 30 minutes to relax and transition into sleep mode. This is crucial for letting your body and mind prepare for rest.
2. **Adequate Sleep:** Aiming for a bedtime of 10:00 PM allows you to potentially get about 7.5 hours of sleep, which can help address some of your sleep deficit. Even if you don’t fall asleep right away, this window gives you the chance to rest and recharge.
3. **Morning Preparation:** Waking up around 7:30 AM will give you an hour to ease into your day before your meeting, allowing you to feel more alert and prepared.

### Suggested Bedt

In [ ]:
from datetime import datetime, timedelta
from langchain.tools import tool
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(api_key=key, model="gpt-4o-mini", temperature=0.5)

@tool
def avoidance_timing(bedtime: str) -> str:
    """
    Uses an LLM to provide personalized guidance on when to avoid caffeine, screen time, and heavy exercise
    based on the user's bedtime.
    """
    try:
        bed = datetime.strptime(bedtime, "%H:%M")
    except ValueError:
        return "Invalid time format. Please provide bedtime as HH:MM in 24-hour format."

    screen_off = (bed - timedelta(hours=1)).strftime('%I:%M %p')
    last_caffeine = (bed - timedelta(hours=6)).strftime('%I:%M %p')
    no_exercise = (bed - timedelta(hours=2)).strftime('%I:%M %p')

    prompt = f"""
You are a helpful sleep coach. A user plans to sleep at {bedtime} (24-hour format).

Please provide a warm, friendly explanation about when they should:
- Avoid caffeine (last time)
- Turn off screens
- Avoid heavy exercise

Include the specific times:
- Caffeine cutoff: {last_caffeine}
- Screen off: {screen_off}
- No heavy exercise: {no_exercise}

Explain why these timings help improve sleep quality and suggest gentle lifestyle tips.
Keep the tone supportive and motivating.
Return your answer as a short paragraph.
"""

    response = llm.invoke(prompt)
    return response.content.strip()


In [ ]:
print(avoidance_timing.run({"bedtime": "22:30"}))


To help you prepare for a restful night’s sleep at 22:30, it's best to make a few adjustments earlier in the evening. Try to avoid caffeine after 16:30, as it can linger in your system and disrupt your ability to fall asleep. Turning off screens by 21:30 allows your mind to unwind and reduces exposure to blue light, which can interfere with melatonin production. Also, aim to finish any heavy exercise by 20:30; while physical activity is great for sleep, vigorous workouts too close to bedtime can leave you feeling energized instead of relaxed. Embrace gentle activities in the evening, like reading or stretching, to create a calming bedtime routine. You've got this, and these small changes can lead to a more refreshing and restorative sleep!


In [ ]:
from datetime import datetime
from langchain.tools import tool
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(api_key=key, model="gpt-4o-mini", temperature=0.7)

@tool
def sleep_checkin(feeling: str, woke_up_time: str) -> str:
    """
    Uses an LLM to analyze how the user felt upon waking and provide personalized advice.
    - feeling: user's description of how they feel (e.g., tired, refreshed, groggy)
    - woke_up_time: time they woke up (24-hour HH:MM)
    """
    try:
        woke_time_obj = datetime.strptime(woke_up_time, "%H:%M")
        woke_time_str = woke_time_obj.strftime("%I:%M %p")
    except ValueError:
        return "Invalid time format. Please provide woke-up time as HH:MM (24-hour format)."

    prompt = f"""
You are a supportive sleep coach.

A user woke up at {woke_time_str} feeling "{feeling}".

Please provide a thoughtful analysis of what this might indicate about their sleep quality,
possible habits to improve, and encouragement. Keep the tone warm and motivating.
If the feeling is unclear, respond with general tips on sleep hygiene.
"""

    response = llm.invoke(prompt)
    return response.content.strip()


In [ ]:
print(sleep_checkin.run({
    "feeling": "groggy",
    "woke_up_time": "07:15"
}))


Good morning! Waking up feeling groggy can be quite frustrating, but it’s a signal from your body that can help you understand your sleep quality better. Let’s explore a few possibilities together.

1. **Sleep Cycle Disruption**: You may have woken up during a deeper stage of sleep rather than a lighter one. Our sleep cycles typically last about 90 minutes, and waking up at the right moment can make a world of difference in how refreshed you feel. If you’re consistently waking up groggy, it might be worth looking at your sleep patterns.

2. **Sleep Duration**: Consider how many hours of sleep you’re getting each night. Adults generally need between 7 to 9 hours of quality sleep. If you’re not hitting that mark, your body might be feeling the effects. 

3. **Sleep Environment**: Your sleeping environment plays a crucial role in how well you rest. Factors like light, noise, and temperature can impact your sleep quality. Creating a calm and dark environment can help improve your rest.

4.

In [ ]:
from langchain.tools import tool
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(api_key=key, model="gpt-4o-mini", temperature=0.2)

@tool
def respond_to_query(query: str) -> str:
    """
    Uses an LLM to respond to general user queries related to sleep tracking, routines, or reminders.
    - Factual only: No creative, speculative, or medical responses.
    """
    prompt = f"""
You are a sleep assistant that only provides factual and practical information.
Respond to the user's query: "{query}"

Guidelines:
- Only answer questions related to sleep tracking, wind-down routines, or reminders.
- Do NOT provide health, wellness, or medical advice.
- If the question is unclear or outside your scope, politely say so.
- Be concise, direct, and avoid creative storytelling.

Context:
- The user's average sleep in the past 7 days is 6.8 hours.
- The average number of sleep interruptions per night is 2.
- You can set reminders 1 hour before bedtime.
- You can generate a basic wind-down routine for sleep quality types: 'good', 'average', 'poor'.

Now respond:
"""

    response = llm.invoke(prompt)
    return response.content.strip()


In [ ]:
respond_to_query.run("How can I improve my sleep quality?")

'To improve your sleep quality, consider implementing a wind-down routine tailored to your sleep quality type. Based on your average sleep interruptions and duration, here’s a basic routine:\n\n**For Average Sleep Quality:**\n1. **1 hour before bed:** Set a reminder to start winding down.\n2. **Relaxation activities:** Engage in calming activities such as reading, gentle stretching, or listening to soothing music.\n3. **Limit screen time:** Avoid screens (phones, computers, TV) during this hour.\n4. **Create a comfortable environment:** Dim the lights and ensure your sleeping area is cool and quiet.\n\nWould you like me to set a reminder for you?'

In [ ]:
model = ChatOpenAI(api_key=key, model="gpt-4o-mini")

tools = [
    generate_wind_down_routine,
    log_sleep,
    get_sleep_reminders,
    generate_routine_from_history,
    check_sleep_disorder,
    suggest_bedtime,
    build_routine_with_calendar,
    avoidance_timing,
    sleep_checkin,
    respond_to_query
]

sleep_coach_agent = create_react_agent(
    model=model,
    tools=tools,
    prompt=(
        "You are a smart and caring Sleep Coach Agent. Your job is to help users improve their sleep quality "
        "by analyzing their sleep data, generating personalized routines, setting reminders, and detecting any issues "
        "based on sleep patterns.\n\n"
        "🧠 You must ALWAYS reason step-by-step and choose the correct tool to help the user based on their input.\n\n"

        "Here’s how to respond:\n"
        "1. Identify the user’s intent.\n"
        "2. Use the correct tool based on the intent.\n"
        "3. Call the tool with appropriate arguments using the user’s input.\n"
        "4. Return a clean, helpful response directly to the user.\n\n"

        "📦 Here are the tools you have access to:\n"
        "- `generate_wind_down_routine`: Suggests bedtime routines based on sleep quality.\n"
        "- `log_sleep`: Logs the user's sleep session.\n"
        "- `get_sleep_reminders`: Generates reminders like when to stop screen use and begin wind-down.\n"
        "- `generate_routine_from_history`: Uses sleep hours and interruptions to build a new routine.\n"
        "- `check_sleep_disorder`: Predicts possible sleep-related health issues.\n"
        "- `suggest_bedtime`: Calculates optimal sleep time based on desired wake-up.\n"
        "- `build_routine_with_calendar`: Suggests sleep timing based on calendar and sleep debt.\n"
        "- `avoidance_timing`: Suggests when to avoid caffeine, screen, and exercise.\n"
        "- `sleep_checkin`: Lets users log how they feel after waking up and provides tips.\n"
        "- `respond_to_query`: General sleep assistant to answer open-ended questions.\n\n"

        "❗️Important:\n"
        "- Only use tools when needed.\n"
        "- Never fabricate information. Always reason using data or tool outputs.\n"
        "- Respond in a friendly, helpful tone.\n\n"

        "🧾 Always return your final answer as a JSON object in the following format:\n"
        "{\n"
        '  "tool_used": "<tool name>",\n'
        '  "response": "<user-facing answer>"\n'
        "}\n"
    ),
    name="sleep_coach_agent",
)


In [ ]:
from langchain.schema import HumanMessage

response = sleep_coach_agent.invoke({
    "messages": [
        HumanMessage(content="I only slept 5 hours and have a meeting at 8 AM.")
    ]
})

response["messages"]  # This is your JSON string response


[HumanMessage(content='I only slept 5 hours and have a meeting at 8 AM.', additional_kwargs={}, response_metadata={}, id='4e550e4a-b707-4aa9-906b-7e4146449259'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_9oNdHZPsjVbfBB00L2IvvSKD', 'function': {'arguments': '{"next_day_meeting":"08:00","sleep_deficit":2}', 'name': 'build_routine_with_calendar'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 943, 'total_tokens': 971, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BlBYFnCfRCWFYXtxYV3nQCOwD9Cb0', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name='sleep_coach_agent', id='run--8b903359-257a-4e02-bf57-7e6b7ddeee11-

In [ ]:
response["messages"][-1].content

'{\n  "tool_used": "build_routine_with_calendar",\n  "response": "Absolutely! It’s great that you’re prioritizing your sleep, especially with an early meeting tomorrow. Let’s work together to set you up for a restful night.\\n\\n### Suggested Wind-Down Time and Bedtime Routine\\n\\n**Wind-Down Time:** Aim to start winding down by **9:00 PM**. \\n\\n**Bedtime:** Try to be in bed by **10:00 PM**. This gives you a solid hour to relax and prepare for sleep.\\n\\n### Why This Timing is Important\\n\\nGiven that you have an early meeting at 8:00 AM and a current sleep deficit, it’s crucial to maximize your sleep tonight. By aiming for a bedtime of 10:00 PM, you can target around 7 hours of sleep, which is a significant improvement over your recent average. This extra time will help you feel more refreshed and alert for your meeting.\\n\\n### Suggested Bedtime Routine\\n\\n1. **Set the Mood (9:00 PM):** Dim the lights in your living space to signal to your body that it’s time to wind down. Co

In [ ]:
from langchain.schema import HumanMessage
human_msg = HumanMessage(content=(
    "Hey coach, I’ve been feeling really groggy when I wake up around 7:30 AM lately. "
    "Last night I slept about 5.5 hours and woke up 3 times, so please log that. "
    "Also, my sleep quality has been pretty poor overall. Can you help generate a bedtime routine for me? "
    "And what time should I turn off screens and stop drinking coffee if I plan to sleep at 10:30 PM? "
    "I’ve averaged about 5.6 hours of sleep and 2.8 interruptions this week. "
    "Sometimes it takes me over 45 minutes to fall asleep, and I wake up multiple times—do you think that indicates a disorder? "
    "I want to wake up tomorrow at 6:45 AM — what time should I go to bed? "
    "Also, I have an important meeting at 7:30 AM and I’m running on a sleep deficit of about 3 hours. "
    "Lastly, how did I sleep overall this week?"
))

response = sleep_coach_agent.invoke({
    "messages": [
        human_msg
    ]
})

response["messages"][-1].content  # This is your JSON string response


'{\n  "tool_used": "multi_tool_use.parallel",\n  "response": {\n    "sleep_logged": "Your sleep session has been logged from 10:30 PM to 7:30 AM.",\n    "bedtime_routine": "Absolutely! Creating a nighttime wind-down routine tailored for someone experiencing poor sleep quality can make a significant difference in how they feel at bedtime. Let’s craft a soothing and engaging routine that gently guides you into a restful night.\\n\\n---\\n\\n**Your Personalized Nighttime Wind-Down Routine**\\n\\n**7:30 PM – Create a Cozy Atmosphere**  \\nAs the sun begins to set, let’s start by transforming your space into a calming sanctuary. Dim the lights or light a few candles with soft, warm hues. Consider playing some gentle instrumental music or nature sounds—perhaps the soothing rustle of leaves or soft ocean waves. This will help signal to your mind and body that it’s time to unwind.\\n\\n**8:00 PM – Mindful Movement**  \\nEngage in a gentle, mindful movement practice. This could be a 15-minute s

In [ ]:
from langchain.schema import HumanMessage

human_msg = HumanMessage(content=(
    "Hey sleep coach, I’ve been feeling groggy when I wake up, especially around 7:30 AM lately. "
    "Last night I went to bed at 11:00 PM and woke up at 7:30 AM, but I had 3 interruptions. Can you log that for me? "
    "I think my sleep quality has been poor — can you generate a good wind-down routine for me tonight? "
    "Also, I usually go to bed at 10:30 PM — when should I stop using screens, exercising, or drinking coffee? "
    "This week, my average sleep has been 5.6 hours with around 2.8 interruptions per night. Can you help create a new routine from that? "
    "It also takes me over 45 minutes to fall asleep most nights and I wake up frequently. Could this be a sleep disorder? "
    "I need to wake up at 6:45 AM tomorrow — what time should I go to sleep? "
    "And I’ve got a big meeting at 7:30 AM with about 3 hours of sleep deficit — when should I wind down tonight? "
    "Please check me in as feeling tired today and help me understand how I’ve been sleeping overall this week. "
    "Also, please remind me when to start winding down and turn off screens before bed."
))


In [ ]:
response = sleep_coach_agent.invoke({
    "messages": [
        human_msg
    ]
})

response["messages"][-1].content  # This is your JSON string response


'```json\n{\n  "tool_used": "multi_tool_use.parallel",\n  "response": {\n    "sleep_logged": "Your sleep from 11:00 PM to 07:30 AM has been successfully logged.",\n    "wind_down_routine": "### Your Personalized Nighttime Wind-Down Routine\\n\\n1. **Create a Cozy Atmosphere**: Dim the lights and use calming scents.\\n2. **Gentle Movement**: Try calming yoga or stretching.\\n3. **Mindful Journaling**: Reflect on your day and express gratitude.\\n4. **Creative Expression**: Engage in a relaxing creative activity.\\n5. **Soothing Soundscapes**: Listen to calming music or nature sounds.\\n6. **Herbal Tea Ritual**: Sip on calming herbal tea.\\n7. **Digital Detox**: Turn off screens and connect with yourself.\\n8. **Guided Visualization**: Picture a peaceful scene in your mind.\\n9. **Bedtime Affirmations**: Repeat calming affirmations as you lie down.\\n10. **Drift into Sleep**: Focus on your breath and relax.\\n\\nFeel free to adjust this to fit your needs!",\n    "avoidance_timing": {\n  